In [108]:
# System packages
import os
import sys
import warnings
# Data related
import numpy as np 
import pandas as pd 

# sklearn tools 
from sklearn.metrics import  accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.preprocessing import FunctionTransformer
# sklearn models
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
import xgboost as xgb
from xgboost import XGBClassifier

In [107]:
!pip3 install --upgrade git+https://github.com/scikit-learn-contrib/categorical-encoding

  Cloning https://github.com/scikit-learn-contrib/categorical-encoding to /private/var/folders/w9/110177p12dsgs8fkfpx2h51m0000gn/T/pip-req-build-h2x7zkzm
    100% |████████████████████████████████| 10.5MB 2.5MB/s ta 0:00:011
    100% |████████████████████████████████| 9.6MB 2.1MB/s ta 0:00:011
    100% |████████████████████████████████| 235kB 3.0MB/s ta 0:00:01
  Using cached https://files.pythonhosted.org/packages/cd/c1/50a758e8247561e58cb87305b1e90b171b8c767b15b12a1734001f41d356/joblib-0.13.2-py2.py3-none-any.whl
  Running setup.py bdist_wheel for category-encoders ... done
  Stored in directory: /private/var/folders/w9/110177p12dsgs8fkfpx2h51m0000gn/T/pip-ephem-wheel-cache-3j97q28d/wheels/94/ac/6d/fe3feae87e68e96dd9d4ab262c1858312af15d4076fd23656f
Successfully built category-encoders


In [109]:
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 50)

In [110]:
# Import custom functions
from utils_functions import *

## 1. Load data

### Use a sample data

In [1]:
sample = pd.read_csv('../data/processed/train_variants_text_sample.csv')
sample.head(1)


NameError: name 'pd' is not defined

In [113]:
sample = sample.dropna(axis = [0,1], how = 'all')

In [114]:
print (sample.notnull().any(axis = 0))

Class        True
Gene         True
Variation    True
Text         True
dtype: bool


In [115]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import FeatureUnion, Pipeline 
from sklearn.feature_extraction.text import TfidfVectorizer


In [124]:
#Custom Transformer that extracts columns passed as argument to its constructor 
class FeatureSelector( BaseEstimator, TransformerMixin ):
    #Class Constructor 
    def __init__( self, feature_names ):
        self._feature_names = feature_names 
    
    #Return self nothing else to do here    
    def fit( self, X, y = None ):
        return self 
    
    #Method that describes what we need this transformer to do
    def transform( self, X, y = None ):
        return X[ self._feature_names ]

In [154]:
#Categrical features to pass down the categorical pipeline 
cateforical_features = ['Gene']

#Numerical features to pass down the numerical pipeline 
text_features = ['Text']

#Defining the steps in the categorical pipeline 
categorical_pipeline = Pipeline( steps = [ ( 'cat_selector', FeatureSelector(categorical_features) ),
                                  
                                  
                                  ( 'one_hot_encoder', OneHotEncoder( sparse = True ) ) ] )
    
#Defining the steps in the numerical pipeline     
text_pipeline = Pipeline( steps = [ ( 'text_selector', FeatureSelector(text_features) ),                                  
                                  ( ('vect', TfidfVectorizer()) )] )


In [155]:
full_pipeline = FeatureUnion( transformer_list = [ #( 'categorical_pipeline', categorical_pipeline ), 
                                                  
                                                  ( 'text_pipeline', text_pipeline ) ] )

In [149]:
from sklearn.model_selection import train_test_split

#Leave it as a dataframe becuase our pipeline is called on a 
#pandas dataframe to extract the appropriate columns, remember?
X = sample.drop('Class', axis = 1)
#You can covert the target variable to numpy 
y = sample['Class'].values 

X_train, X_test, y_train, y_test = train_test_split( X, y , test_size = 0.2 , random_state = 42 )


In [150]:
X_test.head()

,Gene,Variation,Text
281,PTPRD,Deletion,Tyrosine phosphorylation plays a critical role...
286,TGFBR2,P525L,TbR-II (transforming growth factor b [TGF-b] t...
473,PIK3CA,E545G,"Introduction In 2004, Samuels et al. reported..."
227,SMARCA4,R1189*,Rhabdoid tumors of early infancy are highly ag...
436,MLH1,R725C,Identification of a high-risk disease-causing ...


In [151]:

#The full pipeline as a step in another pipeline with an estimator as the final step
full_pipeline_m = Pipeline( steps = [ ( 'full_pipeline', full_pipeline),
                                  
                                  ( 'model', XGBClassifier(objective="multi:softprob", random_state=42) ) ] )



In [152]:
full_pipeline_m

Pipeline(memory=None,
     steps=[('full_pipeline', FeatureUnion(n_jobs=None,
       transformer_list=[('text_pipeline', Pipeline(memory=None,
     steps=[('text_selector', FeatureSelector(feature_names=None)), ('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>...
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1))])

In [153]:
#Can call fit on it just like any other pipeline
clf = full_pipeline_m.fit( X_train, y_train )


XGBoostError: b'[23:34:22] src/objective/multiclass_obj.cu:60: Check failed: preds.Size() == (static_cast<size_t>(param_.num_class) * info.labels_.Size()) SoftmaxMultiClassObj: label size and pred size does not match\n\nStack trace returned 7 entries:\n[bt] (0) 0   libxgboost.dylib                    0x0000001a19368d3c dmlc::StackTrace(unsigned long) + 492\n[bt] (1) 1   libxgboost.dylib                    0x0000001a19368a04 dmlc::LogMessageFatal::~LogMessageFatal() + 52\n[bt] (2) 2   libxgboost.dylib                    0x0000001a193e507a xgboost::obj::SoftmaxMultiClassObj::GetGradient(xgboost::HostDeviceVector<float> const&, xgboost::MetaInfo const&, int, xgboost::HostDeviceVector<xgboost::detail::GradientPairInternal<float> >*) + 442\n[bt] (3) 3   libxgboost.dylib                    0x0000001a19364daf xgboost::LearnerImpl::UpdateOneIter(int, xgboost::DMatrix*) + 1071\n[bt] (4) 4   libxgboost.dylib                    0x0000001a193830ac XGBoosterUpdateOneIter + 172\n[bt] (5) 5   libffi.

In [ ]:

#Can predict with it like any other pipeline
y_pred = full_pipeline_m.predict( X_test ) 


In [ ]:

#Combining numerical and categorical piepline into one full big pipeline horizontally 
#using FeatureUnion
full_pipeline = FeatureUnion( transformer_list = [ ( 'categorical_pipeline', categorical_pipeline ), 
                                                  
                                                  ( 'text_pipeline', text_pipeline ) ] )

## 3. Training

In [28]:
target = sample.Class.values
features = sample[['Gene', 'Variation','Text']].copy()

In [44]:
get_text_data = FunctionTransformer(lambda x: x['Text'], validate=False)


In [45]:
get_group_data = FunctionTransformer(lambda x: x[['Gene','Variation']], validate=False)


In [46]:
process_and_join_features = Pipeline([
    ('features', FeatureUnion([
            ('group_features', Pipeline([
                ('selector', get_group_data),
                ('vec',OneHotEncoder())
            ])),
             ('text_features', Pipeline([
                ('selector', get_text_data),
                ('vec', CountVectorizer())
            ]))
         ])),
    ('clf', XGBClassifier(objective="multi:softprob", random_state=42))
])

In [47]:
process_and_join_features

Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=None,
       transformer_list=[('group_features', Pipeline(memory=None,
     steps=[('selector', FunctionTransformer(accept_sparse=False, check_inverse=True,
          func=<function <lambda> at 0x1a36e931e0>, inv_kw_args=None,
          inverse_func=None, kw_...
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1))])

In [49]:
model = make_pipeline(preprocess)

In [50]:
model

Pipeline(memory=None,
     steps=[('columntransformer', ColumnTransformer(n_jobs=None, remainder='drop', sparse_threshold=0.3,
         transformer_weights=None,
         transformers=[('countvectorizer', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8...nizer=None, vocabulary=None), Gene         True
Variation    True
Text         True
dtype: bool)]))])

In [52]:
clf =model.fit(X_train, y_train)


In [53]:
predicted = clf.predict(X_val)
acc=np.mean(predicted == y_val)
print(acc)

AttributeError: 'ColumnTransformer' object has no attribute 'predict'

In [ ]:
process_and_join_features = Pipeline([
             ('text_features', Pipeline([
                ('selector', get_text_data),
                ('vec', CountVectorizer())
            ]))
         ])),
    ('clf', OneVsRestClassifier(LogisticRegression()))
])

In [27]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.impute import SimpleImputer

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer, make_column_transformer

In [30]:
X_train, X_test, y_train, y_test = train_test_split(features, target, random_state=42)

In [32]:
categorical_features = features.dtypes == 'O'

In [41]:
preprocess = make_column_transformer(
    (categorical_features,CountVectorizer()))

In [42]:
model = make_pipeline(
    preprocess,
    LogisticRegression())

In [43]:
model.fit(X_train, y_train)
print("logistic regression score: %f" % model.score(X_test, y_test))

ValueError: Found input variables with inconsistent numbers of samples: [3, 498]

In [22]:
preprocess = make_column_transformer(
    (['Text','Gene', 'Variation'],  CountVectorizer(preprocessor=clean_text_stemmed))   
)

In [ ]:
preprocess = make_column_transformer(
    (categorical_features, OneHotEncoder()))

## 3.1 Bag of words + Model

### 3.1.3 Feature union + xgboost

In [14]:
from sklearn.pipeline import FeatureUnion

In [83]:
features = FeatureUnion([
                ('Gene_tfidf', 
                  Pipeline([('vect',
                             CountVectorizer(preprocessor=clean_text_stemmed))
                            ])),
                ('variant_tfidf', 
                  Pipeline([('extract_field', 
                             CountVectorizer(preprocessor=clean_text_stemmed))
                            ])),
    ('text_tfidf', 
                  Pipeline([('vect', CountVectorizer(preprocessor=clean_text_stemmed, stop_words =stop_words)),
                ('tfidf', TfidfTransformer()) 
                           ]))
  
    
]) 

In [84]:
pipeline = Pipeline([('features',features),
                     ('clf', XGBClassifier(objective="multi:softprob", random_state=42))])

In [85]:
clf.fit(X_tr, y_tr)  
predicted = clf.predict(X_val)
acc=np.mean(predicted == y_val)
print(acc)

ValueError: DataFrame.dtypes for data must be int, float or bool.
                Did not expect the data types in fields Gene, Variation, Text

In [45]:
X=transformer.transform(sample).toarray()

In [48]:
print(X.shape)

(7, 18)


In [47]:
clf = Pipeline([('clf',  xgb.XGBClassifier(objective="multi:softprob", random_state=42))])                           

In [ ]:
transformer = FeatureUnion([
                ('Gene_tfidf', 
                  Pipeline([('extract_field',
                              FunctionTransformer(lambda x: x['search_term'], 
                                                  validate=False)),
                            ('tfidf', 
                              TfidfVectorizer())])),
                ('product_title_tfidf', 
                  Pipeline([('extract_field', 
                              FunctionTransformer(lambda x: x['product_title'], 
                                                  validate=False)),
                            ('tfidf', 
                              TfidfVectorizer())]))]) 

In [56]:
xgb = XGBClassifier(objective="multi:softprob", random_state=42)


In [ ]:

print(rmspe(combined_data.Sales, np.expm1(pred)))

In [ ]:

pred = cross_val_predict(make_pipeline(processing_pipeline, est), 
                         combined_data, 
                         np.log1p(combined_data.Sales),
                         cv=5)

print(rmspe(combined_data.Sales, np.expm1(pred)))

In [40]:
clf = Pipeline([('vect', MeanEmbeddingVectorizer(w2vec)),
                ('clf',  xgb.XGBClassifier(objective="multi:softprob", random_state=42))])                           

In [41]:
clf.fit(X_tr, y_tr)  
predicted = clf.predict(X_val)
acc=np.mean(predicted == y_val)
print(acc)

0.4626865671641791


In [87]:
# Create the pipeline for the tweets
text = Pipeline([('vect', CountVectorizer(preprocessor=clean_text_stemmed, stop_words =stop_words)),
                ('tfidf', TfidfTransformer())])

# Create the pipeline for the other variables and add selection to choose features
dummies = Pipeline([('vect', CountVectorizer(columns=['Gene', 'Variation']))])

# Merge pipelines using FeatureUnion
features = FeatureUnion([('text', text),
                         ('dummies', dummies)])


TypeError: __init__() got an unexpected keyword argument 'columns'

In [ ]:

# Pipeline with the classifier
pipeline = Pipeline([('features',features),
                     ('clf', SVC(random_state=0, kernel='linear'))])